# Qwen 0.5b on GRPO

---
이 노트북은 [will brown](https://x.com/willccbb)이 작성한 [GRPO 데모](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb)의 변형 버전으로, gsm8k 수학 데이터셋을 사용하여 llama-1b를 학습시키는 내용입니다.

우리는 Colab에서 더 잘 작동하도록 다음과 같은 변경사항들을 구현했습니다:
* llama-1b를 Qwen-0.5b로 교체
* vllm을 사용한 생성 방식 도입(상당한 속도 향상). Qwen의 작은 크기 덕분에 GRPO에 사용되는 것과 동일한 GPU에서 vllm을 실행할 수 있습니다
* flash-attn 제거 (Qwen 모델링에서 반복적인 버그 발생, 원인 불명확)

---
Ref:
- [Original Notebook](https://colab.research.google.com/drive/1bfhs1FMLW3FGa8ydvkOZyBNxLYOu0Hev?usp=sharing#scrollTo=Q7qTZbUcg5VD)
---


## 1. 환경 설정
- 시작을 위해서 여기 가이드 보세요: [Setup Guide](../setup/README.md)
- 이후에 아래 셀 실행을 통해서, 필요한 패키지가 설치 되었는지를 확인 합니다.

In [1]:
! pip list | grep -E "sagemaker|vllm|trl|datasets"

datasets                           3.2.0
sagemaker                          2.232.2
sagemaker-core                     1.0.21
sagemaker-mlflow                   0.1.0
trl                                0.14.0
vllm                               0.7.2


In [2]:
%load_ext autoreload
%autoreload 2

## 2. train.py 테스트 실행
- train.py 파일이 기본적으로 동작 테스트를 로컬에서 합니다. 문법, 데이터 확인 등의 기본적인 작업을 하고, 이후에 SageMaker Training Job 을 통해서 최종 확인 합니다. 

In [5]:
# %%sh
# python scripts/train.py

## 3. 훈련 작업 생성하기

실제 [SageMaker CreateTrainingJob API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html)는 SageMaker Python SDK의 [상위 레벨 'Estimator' 클래스](https://sagemaker.readthedocs.io/en/stable/overview.html)가 단순화하는 데 도움이 되는 여러 저수준 세부 사항을 필요로 합니다. 특히:
- 정확한 컨테이너 이미지 URI를 지정하는 대신, SDK가 선택된 프레임워크와 버전을 기반으로 이를 찾아줍니다
- SDK는 우리의 `scripts` 번들을 자동으로 압축하여 S3에 업로드하고, 훈련 작업이 거기서 로드하도록 구성합니다

따라서 먼저, 작업을 구성하고 어떤 인프라(컴퓨팅 인스턴스의 수와 유형)에서 실행되어야 하는지를 지정하는 `estimator` 객체를 생성합니다:

> ℹ️ 다른 작업을 대신 실행하는 서비스들처럼, 훈련 작업은 S3의 입력 훈련 데이터와 같은 리소스에 접근할 수 있도록 [IAM 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)을 가정합니다. SageMaker 노트북 자체가 이미 가정된 역할로 실행되고 있으므로, 단순화를 위해 훈련 작업 역할을 노트북 역할과 동일하게 설정할 것입니다.


In [6]:
import sagemaker
from sagemaker.pytorch import PyTorch

nb_role = sagemaker.get_execution_role()

estimator = PyTorch(
    source_dir="scripts",
    entry_point="train.py",
    framework_version="2.1.0",
    py_version="py310",
    base_job_name="qrpo-qwen",
    instance_count=1,
    instance_type="ml.p4d.24xlarge",
    keep_alive_period_in_seconds=3600,
    role=nb_role,
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


구성이 완료되면, 입력 데이터 위치를 지정하고 `estimator.fit()`을 실행하여 실제 훈련 작업을 시작할 수 있습니다.

데이터 입력 "채널"의 수, 이름 및 유형은 [사용자가 결정](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html#sagemaker-CreateTrainingJob-request-InputDataConfig)할 수 있습니다: 단지 노트북이 스크립트에서 기대하는 것과 동일한 채널을 구성하도록 하면 됩니다.

In [7]:
%%time

estimator.fit(
    wait=True,  # Wait for the training to complete (default=True)
    logs=True,  # Stream training job logs to the notebook (default=True, requires wait=True)
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: qrpo-qwen-2025-02-09-11-01-47-356


2025-02-09 11:01:52 Starting - Starting the training job...
2025-02-09 11:02:01 Downloading - Downloading the training image
2025-02-09 11:02:01 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2025-02-09 11:02:28,347 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-02-09 11:02:28,474 sagema

UnexpectedStatusException: Error for Training job qrpo-qwen-2025-02-09-11-01-47-356: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "ImportError: /opt/conda/lib/python3.10/site-packages/torch/lib/../../nvidia/cusparse/lib/libcusparse.so.12: undefined symbol: __nvJitLinkComplete_12_4, version libnvJitLink.so.12"
Command "/opt/conda/bin/python3.10 train.py", exit code: 1. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html